# Retinanet Deployment and Video Inference 

Once you have wrapped Retinanet and pushed your container to Dockerhub you are ready to create the deployment and run inference.

## Create deployment

In [ ]:
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, BatchJobsApi, BatchDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator
import requests

SD_IP = ""
username = ""
password = ""

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"

config.oidc_client_id = "sd-api"
config.oidc_client_secret = "sd-api-secret"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"

def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate(username, password)
    api_client = ApiClient(config)
    return api_client

Create the deployment manifest, replacing `CONTAINER_IMAGE` with your own container image name:

In [ ]:
CONTAINER_IMAGE = ""
DEPLOYMENT_NAME = "retinanet"
NAMESPACE = "test"

CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "1"
MEMORY_LIMITS = "2Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "image": CONTAINER_IMAGE,
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

Launch your deployment: 

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

## Copy video to MinIO

We will now copy our video data to MinIO installed in the K8s cluster running Seldon. If you have MinIO installed, open up a seperate terminal and port-forward to MinIO with the following command:

`!kubectl port-forward -n minio-system svc/minio 8090:9000`

Import test video: 

In [ ]:
import torchvision 
import matplotlib.pyplot as plt
frames = torchvision.io.read_video(filename= "input/video1.mp4")
frames[0].size()

We observe the video contains 362 frames of size 540x960. The frames consist of several people riding motorcycles:

In [ ]:
plt.imshow(frames[0][0])

## Batch job

We will now test batch on a smaller range of frames (10 frames). Tensors will be saved to a text file which will be pushed to MinIO:

In [ ]:
textfile = open("images.txt", "w")
for frame in frames[0][0:9]:
    textfile.write(str(frame.tolist()) + "\n")
textfile.close()

In [ ]:
!mc cp images.txt minio/data/

Run batch inference, writing the results to `output-images-{{workflow.name}}.txt`:

In [ ]:
WORKFLOW = {
    "batchDataType": "data",
    "batchMethod": "predict",
    "batchRetries": "3",
    "batchTransportProtocol": "rest",
    "batchWorkers": "15",
    "inputData": "s3://data/images.txt",
    "objectStoreSecretName": "seldon-job-secret",
    "outputData": "s3://data/output-images-{{workflow.name}}.txt"
}

In [ ]:
batch_api = BatchJobsApi(auth())
batch_api.create_seldon_deployment_batch_job(name=DEPLOYMENT_NAME, namespace=NAMESPACE, workflow=WORKFLOW)

In [ ]:
JOB_NAME='<workflowName>'

batch_api = BatchJobsApi(auth())
batch_api.get_deployment_batch_job(name=DEPLOYMENT_NAME, namespace=NAMESPACE, job_name=JOB_NAME)

In [1]:
!mc cat minio/data/output-images-<workflowName>.txt

{"data": {"names": ["t:0", "t:1"], "ndarray": [[[183, 305, 263, 420], [893, 68, 947, 158]], ["person", "person"]]}, "meta": {"requestPath": {"retinanet-container": "seangreaves/pytorch:0.1"}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[[195, 301, 266, 417], [184, 344, 281, 451], [893, 68, 946, 158]], ["person", "motorcycle", "person"]]}, "meta": {"requestPath": {"retinanet-container": "seangreaves/pytorch:0.1"}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[[206, 292, 279, 400], [195, 343, 295, 445], [894, 68, 945, 159]], ["person", "motorcycle", "person"]]}, "meta": {"requestPath": {"retinanet-container": "seangreaves/pytorch:0.1"}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[[213, 287, 283, 396], [204, 334, 302, 437], [891, 70, 937, 156]], ["person", "motorcycle", "person"]]}, "meta": {"requestPath": {"retinanet-container": "seangreaves/pytorch:0.1"}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[[221, 281, 285, 391], [211, 319, 308, 429], 